In [2]:
%matplotlib inlinea

In [3]:
import pandas as pd

In [11]:
climb = pd.read_pickle('../scraping/test_data/_climb')

In [12]:
vocab_location = '/Users/adam/RouteRobot/scraping/vocab.txt'
vocab = pd.read_csv(vocab_location, header=None)[0].tolist()

In [13]:
import numpy as np

In [16]:
cmb = climb.iloc[0]
cmb['sparse_tfidf'].todense()

matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [17]:
def combine_matrix(cmb, route_description_weight = 1.618):
    """ Add sparse representations of route and area descriptions together """
    
    if pd.isnull(cmb['parent_sparse_tfidf']):
        if pd.isnull(cmb['sparse_tfidf']):
            return float('NaN')
        else:
            return cmb['sparse_tfidf']
    else:
        area = np.array(cmb['parent_sparse_tfidf'].todense())
        route = np.array(cmb['sparse_tfidf'].todense()) * route_description_weight
        added = area + route
        # L2 normalize again
        text_vector = added / norm(added, 2)
        return csr_matrix(text_vector)

In [19]:
from scipy.linalg import norm
from scipy.sparse import csr_matrix
climb.apply(combine_matrix, axis=1)

105708962      (0, 2737)\t0.173051326107\n  (0, 2736)\t0.13...
105718711      (0, 37)\t0.182443060869\n  (0, 108)\t0.10999...
105718771      (0, 89)\t0.228553873599\n  (0, 137)\t0.18316...
105738019      (0, 0)\t0.0588036851687\n  (0, 3)\t0.0930428...
105738022      (0, 0)\t0.0611109882915\n  (0, 8)\t0.0584010...
105738025      (0, 83)\t0.0588035216612\n  (0, 88)\t0.05765...
105738028      (0, 3)\t0.118968763599\n  (0, 137)\t0.239816...
105738031      (0, 11)\t0.128489088824\n  (0, 42)\t0.076926...
105738034      (0, 0)\t0.0592546475342\n  (0, 8)\t0.0566270...
105738040      (0, 0)\t0.059802839263\n  (0, 8)\t0.05715093...
105738043      (0, 15)\t0.0713729635833\n  (0, 87)\t0.06933...
105738047      (0, 0)\t0.0589695178429\n  (0, 8)\t0.0563545...
105738050      (0, 303)\t0.116815005687\n  (0, 334)\t0.1389...
105738053      (0, 18)\t0.117681494463\n  (0, 195)\t0.06285...
105738056      (0, 11)\t0.0778628382375\n  (0, 42)\t0.04661...
105738059      (0, 11)\t0.0793604478316\n  (0, 42)\t0.0

In [14]:
if 
area = np.array(cmb['parent_sparse_tfidf'].todense())
route = np.array(cmb['sparse_tfidf'].todense()) * route_description_weight
added = area + route
# L2 normalize again
text_vector = added / norm(added, 2)
return csr_matrix(text_vector)

AttributeError: 'float' object has no attribute 'todense'

In [67]:
import re

In [68]:
state_pattern = re.compile(r'(?:A[KLRZ]|C[AOT]|D[CE]|FL|GA|HI|I[ADLN]|K[SY]|LA|M[ADEINOST]|N[CDEHJMVY]|O[HKR]|PA|RI|S[CD]|T[NX]|UT|V[AT]|W[AIVY])')

In [69]:
isa_state_pattern.search('HI')

In [70]:
state_table = pd.read_csv('../scraping/state_table.csv', index_col=0)

def index_climbs_by_state(climb):
    """ Make filtering by state faster by pre-indexing """
    state_index = {}
    for state in state_table['abbreviation']:
        proper_state_name = state_table['name'][state_table['abbreviation'] == state].iloc[0]
        loc = climb['state_name'] == proper_state_name
        state_index[state] = loc
    return state_index
state_index = index_climbs_by_state(climb)
# pickle.dump(state_index, open('state_index','wb'))
    

In [71]:
def detect_geography(tokens, state_pattern):
    states = []
    for tk in tokens:
        if state_pattern.search(tk):
            states.append(tk)
    if len(states) > 0:
        return states

In [82]:
states = detect_geography(['overhung', 'crimps', 'evening', 'AL', 'fwjehfb', 'UT'], state_pattern)

In [84]:
def filter_geography(climb, state_index, states = None):
    """ return boolean column of climbs in those states """
    include = ~climb.any(1)
    if states:
        for state in states:
            include = include | state_index[state]
    return include
# filter_geography(climb, state_index, states)

In [86]:
climb.dtype()

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [51]:
def create_grade_map(climb, rating_system = ['rateHueco','rateYDS']):
    grade_map = {}
    for label in rating_system:
        grpd = climb.groupby(label)['grade'].mean()
        grade_map.update(dict(zip(grpd.index, grpd.ravel())))
    return grade_map

In [23]:
# create_grade_map(climb)

In [37]:
#def map_grades(climb):
#    grpd = climb.groupby(by=['rateHueco'])
#    grade = grpd.mean()
    #
#    return dict(zip(hueco, grade))

In [ ]:
grade = grpd.mean()
    
    return dict(zip(hueco, grade))

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
def get_sparse_X(text_segments, vocab):
    """ vocab words from description
    returns scipy matrix
    """
    # lemmatize, tokenize, vectorize text
    tfidf = TfidfVectorizer(
        vocabulary=vocab,
        strip_accents = 'unicode', lowercase=True, ngram_range=(1,2),
        norm='l2', sublinear_tf=False, smooth_idf=True, use_idf=True
        )
    X = tfidf.fit_transform(text_segments)
    
    return X

In [63]:
import numpy as np
X = get_sparse_X(climb['description'], vocab)
N = float(X.shape[0])
# nnz = (X != 0).sum(0)[0].tolist()[0]
# calculate IDF
# dump IDF as dictionary
climb['sparse_tfidf'] = [X[i] for i in range(X.shape[0])]


In [88]:
def calculate_idf_weights(X, vocab):
    # count nonzeros by column
    nnz = np.array((X != 0).sum(0))
    # avoids div by zero error
    nnz[nnz == 0] = 1
    # weight by inverse document frequency
    idf = np.log(N / nnz)

    idf_lookup = dict(zip(vocab, idf[0].tolist()))

    return idf_lookup

In [101]:
import pickle
idf_lookup = calculate_idf_weights(X, vocab)
pickle.dump(idf_lookup, open('idf', 'wb'))

In [80]:
print min(nnz[0])
nnz[nnz == 0] = 1
print min(nnz[0])

1
1


In [22]:
len(Df)

2746

In [48]:
# dict(zip(vocab, Df))

In [6]:
#climb[climb['nickname'] == 'genericcrack']
climb[climb.index=='105717367']['keyword']

105717367    weekends  chalked  incredible  corner  form  r...
Name: keyword, dtype: object

In [7]:
LIMIT = 100

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    vocabulary = vocab,
    strip_accents = 'unicode', lowercase=True, norm='l2',
    sublinear_tf=True, smooth_idf=True, use_idf=True
    )
text_segments = climb['description'].tolist()
X = tfidf.fit_transform(text_segments)#[:LIMIT]
#climb = climb[:LIMIT]

In [9]:
X.shape

(69949, 3674)

In [10]:
import pickle
with open(r'tfidf_matrix.pickle', 'wb') as ouput_file:
    pickle.dump(X, ouput_file)

In [11]:
NEED CLIMB INDEX

SyntaxError: invalid syntax (<ipython-input-11-d75a0cedee7a>, line 1)

In [ ]:
import pickle
with open(r'tfidf_matrix.pickle', 'rb') as input_file:
    X = pickle.load(input_file)

In [ ]:
query = dirty_query.strip().lower()
unigrams = query.split(' ')
bigrams = [ b[0]+' '+b[1] for l in [query] for b in zip(l.split(' ')[:-1], l.split(' ')[1:]) ]
tokens = [ t for t in (unigrams + bigrams) if t in vocab ]

In [ ]:
tokens

In [1]:
import math
import numpy as np

def inverse_document_frequency(word, X, vocab):
    """ Given word and tfidf_matrix find its inverse document frequency
    http://nlp.stanford.edu/IR-book/html/htmledition/inverse-document-frequency-1.html
    """
    # count of times all words appear
    nnz = (X != 0).sum(0)[0].tolist()[0]
    # get this word's count
    # max avoids div by zero error
    Df = max(1,nnz[vocab.index(word)])
    # total number of documents in the corpus
    N = X.shape[0]
    idf = math.log(N / Df)
    return idf

def score_tokens(tokens, X, vocab):
    """ Assign tfidf scores to tokens """
    word_scores = list()
    for word in set(tokens):
        idf = inverse_document_frequency(word, X, vocab)
        tf = tokens.count(word)
        word_scores.append(tf * idf)
    zpd = dict(zip(list(set(tokens)), word_scores))
    return zpd

def sanitize(dirty_query):
    """ Preprocess query string """
    query = dirty_query.strip().lower()
    unigrams = query.split(' ')
    bigrams = [ b[0]+' '+b[1] for l in [query] for b in zip(l.split(' ')[:-1], l.split(' ')[1:]) ]
    tokens = [ t for t in (unigrams + bigrams) if t in vocab ]
    return tokens

def score_query(dirty_query, X, vocab):
    """ Find best matching descriptions """
    
    tokens = sanitize(dirty_query)
    # weight tokens by tfidf score
    keyword = score_tokens(tokens, X, vocab)
    
    # represent tokens with a vector in vocabulary space
    text_vector = np.zeros((1,len(vocab)))
    for word, score in keyword.items():
        text_vector[0][vocab.index(word)] = score

    # measure similarity of vector to mp climb descriptions
    cos_sim = (text_vector * X.T)[0]
    scores = (cos_sim / max(cos_sim)).tolist()
    
    return scores

In [2]:
dirty_query = 'fun'
ideal = 0.1

In [47]:
# the loc grammar seems to be import to pandas
display = climb.loc[:,('name','grade','feet','staraverage','starvotes','keyword')] # FUCKING LOCATION

In [48]:
display['text_similarity_score'] = score_query(dirty_query, X, vocab)
# display = display.sort_values('text_similarity_score', ascending=False)

In [49]:
# scores look lognormal
# pd.Series([s for s in scores if s > 0]).hist()

In [50]:
import itertools

def grade_scorer(grade, ideal):
    return 1 - abs(ideal - grade)

def score_grade(grade, ideal):
    grade_score = map(grade_scorer, grade, itertools.repeat(ideal, len(grade)))
    return grade_score

In [51]:
display['grade_score'] = score_grade(grade = display['grade'], ideal = ideal)

In [52]:
display['staraverage_score'] = map(math.log, display['staraverage'])
#display['starvotes_score'] = map(math.log, display['starvotes'])

In [58]:
display['best'] = display[['staraverage_score','grade_score','text_similarity_score']].sum(1)
display = display[display['starvotes'] > 5]

In [24]:
# display.sort_values('best', ascending=False)

In [ ]:
# if col in ['aid','alpine','boulder','sport','trad','ice']:
#     return ideal * grade
# elif col in ['feet', 'pitches', 'grade']:
#     return 1 - abs(ideal - grade)
# elif col in ['staraverage', 'starvotes']:
#     return grade